In [ ]:
#| default_exp chatbot_tab

In [ ]:
#| export
import ipywidgets as widgets
import traitlets
from ipywidgets import Textarea, Text, Layout, HBox, Stack, Layout
from traitlets import HasTraits
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI
#from jupyter_mentor.chatbot import ChatBot, ChatBotModel, ChatBotView
from jupyter_mentor.madlib import MadLibModel, MadLibView
from jupyter_mentor.llm import FileModel

In [ ]:
#|export
class ChatBotModel(FileModel):

    def __init__(self, madlibs_list):
        super().__init__()
        self.human_template = "{input_text}"
        self.bot_template = "You are playing the role of a tutor/educator"
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.update_bot_template(self.bot_template)

        self.madlib_models = []
        for value in madlibs_list:
            model = MadLibModel(**value)
            self.madlib_models.append(model)

    def update_bot_template(self, bot_template):
        self.bot_template = bot_template
        self.bot_message_prompt = SystemMessagePromptTemplate.from_template(self.bot_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])

    def update_human_template(self, human_template):
        self.human_template = human_template
        self.human_message_prompt = HumanMessagePromptTemplate.from_template(self.human_template)
        self.chat_prompt = ChatPromptTemplate.from_messages([self.bot_message_prompt, self.human_message_prompt])        

    def prompt(self, input_text):
        ret = self.llm.invoke(input_text)
        return ret.content
    
    def prompt_with_template(self, kwargs):
        ret = self.llm.invoke(self.chat_prompt.format_prompt(**kwargs))
        return ret.content

In [ ]:
madlibs_list = [
    {'name': 'Open Prompt',
     'description': '',
     'variables': {
         'input_text': 'value'
        }, 
        'template': '{input_text}' ,
        'placeholder': 'Message AI chatbot...',
    },
    {
        'name': 'Metaphor',
        'variables': {
            'concept': 'value'
        }, 
        'template': "I'm having trouble understanding {concept}. Explain it as a metaphor",
        'description': "I'm having trouble understanding...", 
        'placeholder': 'why the sky is blue'
    }
]

In [ ]:
chatbot_model = ChatBotModel(madlibs_list)
chatbot_model.prompt('hello')

In [ ]:
chatbot_model.madlib_models[0].variables

In [ ]:
#|export
class ChatBotView(widgets.VBox):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    
    def __init__(self):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat = Textarea(
            disabled = True,
            layout=Layout(width='90%', height='400px')
        )
        self.radio_buttons = widgets.RadioButtons()
        self.hbox = widgets.HBox()
        self.default = widgets.Text(
            placeholder='Message AI chatbot...',
            #layout=Layout(width='100%')
        )
        self.stack = widgets.Stack(children = (self.default, ))
        self.stack.selected_index = 0
        
        self.submit_button = widgets.Button(
            value=False,
            disabled=False,
            button_style='success',
            icon='arrow-circle-right' 
        )
        self.hbox.children = (self.stack, self.submit_button)
        self.children = (self.chat, self.radio_buttons, self.hbox ) 

In [ ]:
chatbotview = ChatBotView()
chatbotview

In [ ]:
#|export
class ChatBot(ChatBotView):
    """ This view also exists in 04_chatbot, but wea
    are overwriting it here
    """
    selected_index = traitlets.Int()
    
    def __init__(self, model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()
        self.model = model
       
        self.selected_model = 0

        madlib_views = []
        for model in self.model.madlib_models:
            view = MadLibView(model)
            madlib_views.append(view)
            self.stack.children += (view, )

        #self.selected_index = 0
        traitlets.link((self, 'selected_index'), (self.stack, 'selected_index'))

        self.submit_button.on_click(self.on_click)

    def on_click(self, change):
        self.chat.value = self.chat.value + "USER: " + self.stack.children[0].value + '\n\n'
        self.stack.children[0].value = ''
        ret = self.model.prompt(self.stack.children[0].value)
        self.chat.value = self.chat.value +  "CHATBOT: "  + ret + '\n\n'

In [ ]:
chatbot = ChatBot(chatbot_model)
chatbot

In [ ]:
chatbot.selected_index = 0

In [ ]:
#| export
class StudentChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool() 
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.step_by_step = widgets.Button(
            description='Step-by-Step',
            button_style='info',
            tooltip='Description',
        )
        self.metaphor = widgets.Button(
            description='Metaphor',
            button_style='info',
            tooltip='Description',
        )
        self.hints = widgets.Button(
            description='Hints',
            button_style='info',
            tooltip='Description',
        )
        self.guided_questions = widgets.Button(
            description='AI Guided Questions',
            button_style='info',
            tooltip='Description', 
        )
        self.suggestion_buttons.children = (self.step_by_step, self.metaphor, self.hints, self.guided_questions)
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
student_chatbot = StudentChatBot(chatbot_model)
student_chatbot

In [ ]:
#|export
class EducatorChatBot(widgets.VBox):

    #user = traitlets.CUnicode()
    #response = traitlets.CUnicode()
    #step_by_step = traitlets.Bool()
    #metaphor = traitlets.Bool()
    #hints = traitlets.Bool()
    #guided_questions = traitlets.Bool()  
    
    def __init__(self, chatbot_model):
        # If you forget to call the superconstructor on an extended widget
        # you will get an AttributeError: object has no attribute '_model_id'
        super().__init__()

        self.chat_bot = ChatBot(chatbot_model)

        self.suggestion_buttons = HBox()
        self.exam_questions = widgets.Button(
            description='Exam Questions',
            button_style='info',
            tooltip='Description',
        )
        self.lesson_plan = widgets.Button(
            description='Lesson Plan',
            button_style='info',
            tooltip='Description',
        )
        self.suggestion_buttons.children = (self.exam_questions, self.lesson_plan )
        self.children = (self.chat_bot, self.suggestion_buttons)

In [ ]:
educator_chatbot = EducatorChatBot(chatbot_model)
educator_chatbot

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()